In [1]:
import geopandas as gpd
import geoengine as ge

/home/droenner/git/geoengine-python/env/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
ge.initialize("http://localhost:3030/api", credentials=("admin@localhost", "admin"))

In [3]:
session = ge.get_session()
user_id = session.user_id
session

Server:              http://localhost:3030/api
User Id:             d5328854-6190-4af9-ad69-4e74b0961ac9
Session Id:          eeabd4e8-d028-45ee-b238-09aad7237134
Session valid until: 2023-03-27T09:11:26.807Z

In [4]:
used_quota_start = ge.get_quota(user_id)['used']
used_quota_start

0

In [5]:
bounds_array = [421398.30688394164, 5681079.293017321, 476200.5934965983, 5727832.747780239]
xmin = bounds_array[0]
ymin = bounds_array[1]
xmax = bounds_array[2]
ymax = bounds_array[3]

(xmin, ymin, xmax, ymax)

(421398.30688394164, 5681079.293017321, 476200.5934965983, 5727832.747780239)

In [6]:
sentinel2_band_scl_workflow = ge.unstable.workflow_blueprints.sentinel2_band("SCL")
sentinel2_band_scl_workflow.to_workflow_dict()

{'type': 'Raster',
 'operator': {'type': 'GdalSource',
  'params': {'data': {'type': 'external',
    'providerId': '5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5',
    'layerId': 'UTM32N:SCL'}}}}

In [7]:
senttinel2_band_scl_workflow_id = ge.register_workflow(sentinel2_band_scl_workflow.to_workflow_dict())
senttinel2_band_scl_workflow_id

11847314-a330-5360-b27b-50cf9eca6613

In [8]:
from datetime import datetime
time_start = datetime(2021, 1, 1)
time_end = datetime(2021, 12, 31)

study_area = ge.api.RasterQueryRectangle(
    spatialBounds=ge.SpatialPartition2D(xmin, ymin, xmax, ymax).to_api_dict(),
    timeInterval=ge.TimeInterval(time_start, time_end).to_api_dict(),
    spatialResolution=ge.SpatialResolution(10.0, 10.0).to_api_dict(),
)
study_area

{'spatialBounds': {'upperLeftCoordinate': {'x': 421398.30688394164,
   'y': 5727832.747780239},
  'lowerRightCoordinate': {'x': 476200.5934965983, 'y': 5681079.293017321}},
 'timeInterval': {'start': '2021-01-01T00:00:00.000+00:00',
  'end': '2021-12-31T00:00:00.000+00:00'},
 'spatialResolution': {'x': 10.0, 'y': 10.0}}

In [9]:
status_download = {}

for b in ["B02", "B03", "B04", "B08", "SCL"]:
    print(b)
    sentinel2_band_workflow = ge.unstable.workflow_blueprints.sentinel2_band(b)
    sentinel2_band_workflow_id = ge.register_workflow(sentinel2_band_workflow.to_workflow_dict())
    sentinel2_band_workflow_dataset_task = sentinel2_band_workflow_id.save_as_dataset(study_area, f"sentinel2_nrw_crop_10m_{b}")
    sentinel2_band_workflow_dataset_task.wait_for_finish(print_status=False)
    print(sentinel2_band_workflow_dataset_task.get_status())
    status_download[b] = sentinel2_band_workflow_dataset_task.get_status()

status_download

B02
status=completed, time_started=2023-03-27 08:11:27.359000+00:00, info={'dataset': '26351390-fade-4aed-b954-a7ba252a8583', 'upload': 'd8fbf0d0-7c84-40c2-9d33-556faa7a46e4'}, time_total=00:51:11
B03


ConnectionError: HTTPConnectionPool(host='localhost', port=3030): Max retries exceeded with url: /api/tasks/f246543e-6e0c-492f-88f7-0648db7912ca/status (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc0f31e3ee0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
used_quota_download = ge.get_quota(user_id)['used'] - used_quota_start
used_quota_download

Local Sentinel2 to month aggregates workflow


In [ ]:
## IF you ran the workflow, you can get the ids from the status
band_data_map = { name: task_status.info['dataset'] for name, task_status in status_download.items() }
## IF you have the data already added, youn simply define the ids.
#band_data_map = {
#    'b02': '48d30acd-a378-4f2a-89b3-d73430c0f29e',
#    'b03': '200ed06c-fcd1-41a3-b2dd-68b5d0b338a9',
#    'b04': 'b9ececef-5a0c-4294-be16-334e8017f60f',
#    'b08': '8e635367-35ff-42f9-bb21-88ae9db8be8d',
#    'scl': '64636605-10da-4576-bf21-a265cc1f7d9c'
#    }

band_data_id_map = {name.lower(): 
    ge.api.InternalDataId(
        type="internal",
        datasetId=id) for name, id in band_data_map.items()
} 

band_data_id_map

In [ ]:
status_agg = {}

for b in ["B02", "B03", "B04", "B08", "NDVI"]:
    print(b)
    sentinel2_band_workflow = ge.unstable.workflow_blueprints.s2_cloud_free_monthly_band_custom_input(b, band_data_id_map, granularity="months", window_size=1, aggregation_type="mean")
    sentinel2_band_workflow_id = ge.register_workflow(sentinel2_band_workflow.to_workflow_dict())
    sentinel2_band_workflow_dataset_task = sentinel2_band_workflow_id.save_as_dataset(study_area, f"sentinel2_nrw_crop_10m_cf_monthly_{b}")
    sentinel2_band_workflow_dataset_task.wait_for_finish(print_status=False)
    print(sentinel2_band_workflow_dataset_task.get_status())
    status_agg[b] = sentinel2_band_workflow_dataset_task.get_status()

status_agg

In [ ]:
used_quota_agg = ge.get_quota(user_id)['used'] - used_quota_download
used_quota_agg

Points with monthly Sentinel2 data

In [ ]:
monthly_band_data_map = { name: task_status.info['dataset'] for name, task_status in status_agg.items() }

#monthly_band_data_map = {
#    'b02': 'a291c42d-ec5a-4702-954e-a76b17154752',
#    'b03': '8a15acd3-e135-4cc1-83b3-a883a52d69d7',
#    'b04': '9d3f5b09-3cb6-4dac-a673-9e5ead88a221',
#    'b08': 'd7775c4c-ab71-4f8e-8fac-ccedb154861a',
#    'ndvi': '60fb38a6-9002-4254-8554-15ac293876c7'
#    }

monthly_band_data_id_map = {name: 
    ge.api.InternalDataId(
        type="internal",
        datasetId=id) for name, id in monthly_band_data_map.items()
} 


In [ ]:
points_df = gpd.read_file("group_sample_frac1_inspireId_utm32n.gpkg")
points_id = ge.upload_dataframe(points_df, "group_sample_frac1_inspireId")
points_id

In [ ]:
points_source_operator = ge.unstable.workflow_operators.OgrSource(str(points_id))
points_source_operator.to_workflow_dict()

In [ ]:
# projected_points = ge.unstable.workflow_operators.Reprojection(points_source_operator, target_spatial_reference="EPSG:32632") # only needed if input not projected

points_with_s2_cloud_free = ge.unstable.workflow_operators.RasterVectorJoin(
    raster_sources=[ge.unstable.workflow_operators.GdalSource(x) for x in monthly_band_data_id_map.values() ],
    vector_source=points_source_operator, #projected_points,
    new_column_names=[x for x in monthly_band_data_id_map.keys()],
)

points_with_s2_cloud_free.to_workflow_dict()

workflow = ge.register_workflow(points_with_s2_cloud_free.to_workflow_dict())
workflow

In [ ]:
start_dt = datetime(2021, 1, 1, 0, 0, 0)
end_dt = datetime(2022, 1, 1, 0, 0, 0)

start_dt, end_dt

In [ ]:
gp_res = await workflow.vector_stream_into_geopandas(
    ge.QueryRectangle(
        spatial_bounds=ge.BoundingBox2D(
            xmin=xmin,
            ymin=ymin,
            xmax=xmax,
            ymax=ymax,
        ),
        time_interval=ge.TimeInterval(
            start=start_dt,
            end=end_dt,
        ),
        resolution=ge.SpatialResolution(
            10.0,
            10.0,
        ),
        srs="EPSG:32632",
))

gp_res.to_file("gp_res_10_frac1_monthly_utm32n.gpkg", driver="GPKG")
gp_res

In [ ]:
used_quota_rvjoin = ge.get_quota(user_id)['used'] - used_quota_agg
used_quota_rvjoin